In [1]:
# Data-handling libraries
import pandas as pd

# Feature extraction/transformation libraries
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, LassoCV
from sklearn.decomposition import PCA

# Modelling libraries
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

# Random seed
seed = 5777

# Import data

In [3]:
essays = pd.read_csv("../data_private/essays_cleaned.csv")

# Select/extract features

In [4]:
X = essays.drop("score")
y = essays["score"]

KeyError: "['score'] not found in axis"

## Vectorize text

In [8]:
X_cvec = CountVectorizer(
        ngram_range = (1,2),
        min_df = 2, # ignore hapaxes
    ).fit_transform(X[["essay"]]);

X = pd.concat([
    X.drop("essay", axis=1),
    X_cvec
], axis=1)

## Scale all features

In [9]:
X = StandardScalar().fit_transform(X);

## Select feature set

In [10]:
X = PCA(random_state = seed).fit_transform(X);

## Create train & test sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y);

# Fit models

In [11]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)

print("Train score:", gnb.score(X_train, y_train))
print("Test score:", gnb.score(X_test, y_test))

NameError: name 'X_train' is not defined

# Evaluate models